# BA Thesis - Fairness in Machine Learning
---

This `Jupyter`-notebook is here to implement the necessary tasks to fulfill the analysis and so on for this thesis.

## Packages
The following packages are used in this project

In [112]:
! pip install -r ./requirements.txt

## Generel settings

In [211]:
import numpy as np
import pandas as pd

In [212]:
# define a random state
random_state = 12014500

In [213]:
np.random.seed(random_state)

## Dataset

### Load Data

In [530]:
# load the data
df = pd.read_csv('./data/synthetic_data.csv')

In [531]:
display(df.head())

,Unnamed: 0,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,46,Private,522438,13,Never-married,Exec-managerial,Not-in-family,White,0,0,0,49,United-States,0
1,1,72,Private,64571,7,Widowed,Exec-managerial,Not-in-family,White,0,0,0,73,United-States,0
2,2,25,Private,132619,10,Never-married,Craft-repair,Own-child,White,1,0,0,46,United-States,0
3,3,51,Federal-gov,34355,11,Married-civ-spouse,Adm-clerical,Husband,White,1,0,0,40,United-States,0
4,4,31,Private,213603,10,Never-married,Adm-clerical,Other-relative,White,0,0,0,40,United-States,0


### Pre-Processing

In [564]:
from sklearn.model_selection import train_test_split

In [565]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=random_state)

In [566]:
# Replace '?' with NaN, if necessary (check those special characters)
df_train.replace('?', np.nan, inplace=True)
df_test.replace('?', np.nan, inplace=True)

In [567]:
df_train.isnull().any(axis=1).sum()

5853

In [568]:
df_train.isnull().sum()

Unnamed: 0           0
age                  0
workclass         3502
fnlwgt               0
education-num        0
marital-status       0
occupation        3586
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country    1354
income               0
dtype: int64

In [569]:
df_test.isnull().any(axis=1).sum()

1401

In [570]:
df_test.isnull().sum()

Unnamed: 0          0
age                 0
workclass         849
fnlwgt              0
education-num       0
marital-status      0
occupation        890
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    288
income              0
dtype: int64

In [571]:
print("Training-set contains " + str(df_train.duplicated().sum()) + " duplicated observations") 
print("Test-set contains " + str(df_test.duplicated().sum()) + " duplicated observations") 

Training-set contains 0 duplicated observations
Test-set contains 0 duplicated observations


In [572]:
ratio_features = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
ordinal_features = ['education-num'] # 'education-num' is a numerical representation of 'education' ('education' will be removed)
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex'] # 'native-country' will be removed
target = 'income'

In [573]:
df_train.drop_duplicates(inplace=True, ignore_index=True)
df_test.drop_duplicates(inplace=True, ignore_index=True)

In [574]:
columns_to_drop = ["native-country", 'Unnamed: 0']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

## ML-Models

### Baseline for Synthetic Data

In [575]:
from utils import train_and_evaluate
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [576]:
models = {
    'mlp': MLPClassifier(verbose=False, max_iter = 250, random_state=random_state),
    'rf': RandomForestClassifier(n_estimators=200, max_depth=7 ,random_state=random_state),
    'lr': LogisticRegression(max_iter=1000, solver='lbfgs', random_state=random_state)
}

In [587]:
def describe_model(models, results):
    for model_name, model in models.items():
        print(f"model {model.__class__.__name__} with {model.get_params()}")
        print("\tAccuracy: " + str(results[model_name]['accuracy']))
        print("\tF1: " + str(results[model_name]['f1']))
        print("\tPrecision: " + str(results[model_name]['precision']))
        print("\tRecall: " + str(results[model_name]['recall']))
        print("\n")


In [588]:
def evaluate_models(df_train, df_test, nominal_features, target, describe = True):
    results = {}
    for model_name, model in models.items():
        results[model_name] = train_and_evaluate(model, df_train, df_test, nominal_features, target)

    if describe:
        describe_model(models, results)

    return results

In [589]:
results = evaluate_models(df_train, df_test, nominal_features, target)

/Users/maxkleinegger/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.



model MLPClassifier with {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 250, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 12014500, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
	Accuracy: 0.8479885351622479
	F1: 0.7916473045549122
	Precision: 0.8178309362254184
	Recall: 0.7743800267826496


model RandomForestClassifier with {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'rand

In [602]:
from utils import _split_data
from scipy.stats import mode

def compute_pred(df):
    preds = []
    for _, model in models.items():
        preds.append(model.predict(df))

    stacked_vectors = np.vstack(preds)
    return mode(stacked_vectors, axis=0).mode[0]

X_train, _, _, _ = _split_data(df_train, df_test, nominal_features, 'income')

df_train['income-predicted'] = compute_pred(X_train)

/var/folders/5s/7wthnl510p9_gg2ytrfvxy180000gn/T/ipykernel_48938/1100927733.py:10: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



### Skewing attribute
Should ask Prof about this topic, how to and where?

In [440]:
races = set(list(df['race'].values))
probs = [0.01, 0.02, 0.04, 0.01, 0.92]

race_distributions = []
for i, race in enumerate(races):
    race_distributions = race_distributions + ([race] * int(probs[i]*df_train.shape[0]))

race_distributions = race_distributions + (['White'] * (df_train.shape[0] - len(race_distributions)))

np.random.shuffle(race_distributions)
df_train['race'] = race_distributions

#### Performance for skewed data

In [442]:
evaluate_models(df_train, df_test, nominal_features, target)

MLPClassifier(random_state=12014500)

## Fairness-Evaluation
This parts evaluates the fairness aspects of the basic data. Which attributes may be preserved priveleged, etc. Furthermore we calculate all the attributes to assess the fairness aspect

In [604]:
df_train.head()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,income,income-predicted
0,47,Self-emp-inc,334052,13,Married-civ-spouse,Sales,Husband,White,1,0,0,51,1,1
1,35,Local-gov,215409,6,Divorced,Transport-moving,Not-in-family,White,1,7612,0,40,0,0
2,43,Private,395792,14,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,46,1,1
3,52,Private,180461,11,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,49,1,1
4,27,Federal-gov,293395,9,Never-married,Transport-moving,Not-in-family,White,1,0,1729,40,0,0


In [515]:
features_4_scanning = ['race', 'sex']

In [607]:
def print_report(data, subset):
    """Utility function to pretty-print the subsets."""
    if subset:
        to_choose = df_train[subset.keys()].isin(subset).all(axis=1)
        df = df_train[["income", "income-predicted"]][to_choose]
    else:
        for col in features_4_scanning:
            subset[col] = list(data[col].unique())
        df = df_train[["income", "income-predicted"]]

    true = df["income"].sum()
    pred = df["income-predicted"].sum()

    print("\033[1mSubset: \033[0m")
    print(subset)
    print("\033[1mSubset Size: \033[0m", len(df))
    print("\033[1mTrue Clicks: \033[0m", true)
    print("\033[1mPredicted Clicks: \033[0m", pred)
    print()

In [608]:
from aif360.detectors.mdss.generator import get_random_subset

random_subset = get_random_subset(df_train[features_4_scanning], prob = 0.05, min_elements = 10000)
print_report(df_train, random_subset)

Subset: 
{'race': ['White', 'Amer-Indian-Eskimo', 'Black', 'Asian-Pac-Islander', 'Other'], 'sex': [1, 0]}
Subset Size:  39073
True Clicks:  10545
Predicted Clicks:  8092



In [616]:
from aif360.detectors.mdss.ScoringFunctions import Bernoulli
from aif360.detectors.mdss.MDSS import MDSS

# Bias scan
scoring_function = Bernoulli(direction="negative")
scanner = MDSS(scoring_function)

scanned_subset, _ = scanner.scan(
    df_train[features_4_scanning],
    expectations=df_train["income-predicted"],
    outcomes=df_train["income"],
    penalty=1,
    num_iters=1,
    verbose=False,
)

print_report(df_train, scanned_subset)

Subset: 
{'race': ['White', 'Amer-Indian-Eskimo', 'Black', 'Asian-Pac-Islander', 'Other'], 'sex': [1, 0]}
Subset Size:  39073
True Clicks:  10545
Predicted Clicks:  8092



In [619]:
print_report(df_train, {'race': ['White'], 'sex': [0]})
print_report(df_train, {'race': ['White'], 'sex': [1]})
print_report(df_train, {'race': ['Black'], 'sex': [0]})
print_report(df_train, {'race': ['Black'], 'sex': [1]})
print_report(df_train, {'race': ['Amer-Indian-Eskimo'], 'sex': [0]})
print_report(df_train, {'race': ['Amer-Indian-Eskimo'], 'sex': [1]})
print_report(df_train, {'race': ['Asian-Pac-Islander'], 'sex': [0]})
print_report(df_train, {'race': ['Asian-Pac-Islander'], 'sex': [1]})
print_report(df_train, {'sex': [0]})
print_report(df_train, {'sex': [1]})

Subset: 
{'race': ['White'], 'sex': [0]}
Subset Size:  9798
True Clicks:  1799
Predicted Clicks:  1339

Subset: 
{'race': ['White'], 'sex': [1]}
Subset Size:  18291
True Clicks:  7275
Predicted Clicks:  5863

Subset: 
{'race': ['Black'], 'sex': [0]}
Subset Size:  2876
True Clicks:  220
Predicted Clicks:  61

Subset: 
{'race': ['Black'], 'sex': [1]}
Subset Size:  2263
True Clicks:  262
Predicted Clicks:  148

Subset: 
{'race': ['Amer-Indian-Eskimo'], 'sex': [0]}
Subset Size:  609
True Clicks:  59
Predicted Clicks:  22

Subset: 
{'race': ['Amer-Indian-Eskimo'], 'sex': [1]}
Subset Size:  583
True Clicks:  84
Predicted Clicks:  51

Subset: 
{'race': ['Asian-Pac-Islander'], 'sex': [0]}
Subset Size:  1001
True Clicks:  193
Predicted Clicks:  83

Subset: 
{'race': ['Asian-Pac-Islander'], 'sex': [1]}
Subset Size:  2083
True Clicks:  551
Predicted Clicks:  506

Subset: 
{'sex': [0]}
Subset Size:  14711
True Clicks:  2298
Predicted Clicks:  1510

Subset: 
{'sex': [1]}
Subset Size:  24362
True Cl

### Creating a dataset for evaluation

In [701]:
from aif360.datasets import StandardDataset

def convert_to_standard_dataset(df, target_label_name, scores_name=""):

    # List of names corresponding to protected attribute columns in the dataset.
    # Note that the terminology "protected attribute" used in AI Fairness 360 to
    # divide the dataset into multiple groups for measuring and mitigating 
    # group-level bias.
    protected_attributes=['sex', 'race']
    
    # columns from the dataset that we want to select for this Bias study
    selected_features = df.columns
    
    # This privileged class is selected based on MDSS subgroup evaluation.
    # in previous steps. In our case non-homeowner (homeowner=0) are considered to 
    # be privileged and homeowners (homeowner=1) are considered as unprivileged.
    privileged_classes = [[1], ['White']]   

    # Label values which are considered favorable are listed. All others are 
    # unfavorable. Label values are mapped to 1 (favorable) and 0 (unfavorable) 
    # if they are not already binary and numerical.
    favorable_target_label = [1]

    # List of column names in the DataFrame which are to be expanded into one-hot vectors.
    categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship']

    # create the `StandardDataset` object
    standard_dataset = StandardDataset(df=df, label_name=target_label_name,
                                    favorable_classes=favorable_target_label,
                                    scores_name=scores_name,
                                    protected_attribute_names=protected_attributes,
                                    privileged_classes=privileged_classes,
                                    categorical_features=categorical_features,
                                    # features_to_keep=selected_features
                                    )
    if scores_name=="":
        standard_dataset.scores = standard_dataset.labels.copy()
        
    return standard_dataset

In [702]:
# Create two StandardDataset objects - one with true conversions and one with
# predicted conversions.

# First create the predicted dataset
dataset_pred = convert_to_standard_dataset(df_train, target_label_name = 'income-predicted')
# dataset_orig = convert_to_standard_dataset(df_train, target_label_name = 'income')
# Use this to create the original dataset
dataset_orig = dataset_pred.copy()
dataset_orig.labels =  df_train.dropna()["income"].values.reshape(-1, 1)
dataset_orig.scores =  df_train.dropna()["income"].values.reshape(-1, 1)

In [ ]:
# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric


def compute_metrics(
    dataset_true, dataset_pred, unprivileged_groups, privileged_groups, disp=True
):
    """Compute the key metrics"""
    classified_metric_pred = ClassificationMetric(
        dataset_true,
        dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
    )
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5 * (
        classified_metric_pred.true_positive_rate()
        + classified_metric_pred.true_negative_rate()
    )
    metrics["Statistical parity difference"] = (
        classified_metric_pred.statistical_parity_difference()
    )
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = (
        classified_metric_pred.average_odds_difference()
    )
    metrics["Equal opportunity difference"] = (
        classified_metric_pred.equal_opportunity_difference()
    )
    metrics["Theil index"] = classified_metric_pred.theil_index()

    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))

    return metrics

In [717]:
privileged_groups = [{"sex": 1,  'race': 1}]
unprivileged_groups = [{"sex": 0, 'race': 0}]
metrics = compute_metrics(dataset_orig, dataset_pred, unprivileged_groups, privileged_groups)

Balanced accuracy = 0.7736
Statistical parity difference = -0.3101
Disparate impact = 0.1198
Average odds difference = -0.2772
Equal opportunity difference = -0.4673
Theil index = 0.1388


Use BinaryLabelMetric to compare metrics of predicted vs original

In [728]:
from aif360.explainers import MetricTextExplainer

classified_metric = ClassificationMetric(
        dataset_orig, dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
)

explainer_classified_metric = MetricTextExplainer(classified_metric)
print(explainer_classified_metric.disparate_impact())
print(explainer_classified_metric.average_odds_difference())
print(explainer_classified_metric.statistical_parity_difference())
print(explainer_classified_metric.equal_opportunity_difference())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.11975200083842225
Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.27724931870905417
Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.3100520692995215
True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.4672961309896644


In [730]:
from aif360.metrics import MDSSClassificationMetric

mdss_classification_metric = MDSSClassificationMetric(
        dataset_orig, dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
)

## Mitigation